# Phase 5: Decorators, Generators, and Advanced Patterns
## The patterns that separate beginners from professionals

These patterns show up in every serious Python codebase. Odibi uses all of them. 
When you can write a decorator or a generator from scratch in an interview, 
you signal that you are not a beginner.

---
## Section 1: Functions Are Objects

In Python, functions are just objects. You can:
- Store them in variables
- Pass them as arguments to other functions
- Return them from other functions

This concept is the foundation of decorators.

In [ ]:
# Functions are objects -- you can store them in variables
def greet(name):
    return f"Hello, {name}!"

# Store the function (NOT calling it -- no parentheses)
my_func = greet
print(my_func("Odibi"))  # Hello, Odibi!

# Pass a function as an argument
def apply_to_list(items, func):
    """Apply a function to each item in a list."""
    return [func(item) for item in items]

names = ["alice", "bob", "charlie"]
upper_names = apply_to_list(names, str.upper)
print(upper_names)  # ['ALICE', 'BOB', 'CHARLIE']

### Exercise 1.1
Write a function `transform_data(data, transformer)` that takes a list and a function, 
applies the function to each item, and returns the results.
Test it with `str.lower`, `len`, and a custom function.

In [ ]:
# Exercise 1.1
# YOUR CODE HERE


---
## Section 2: Closures

A **closure** is a function that remembers variables from the scope where it was created, 
even after that scope has finished executing.

This is how decorators work under the hood.

In [ ]:
# Closure example -- a function factory
def make_multiplier(factor):
    """Create a function that multiplies by a fixed factor."""
    def multiplier(x):
        return x * factor  # 'factor' is remembered from the outer scope
    return multiplier

double = make_multiplier(2)
triple = make_multiplier(3)

print(double(5))   # 10
print(triple(5))   # 15
print(double(100)) # 200

# Each function remembers its own 'factor' value!

---
## Section 3: Decorators

A **decorator** is a function that wraps another function to add behavior. 
The `@` syntax is just shorthand:

```python
@my_decorator
def my_function():
    pass

# Is exactly the same as:
def my_function():
    pass
my_function = my_decorator(my_function)
```

Odibi's `@transform` decorator in `registry.py` uses this pattern to register 
transform functions automatically.

In [ ]:
import time
from functools import wraps

# Step-by-step: building a @timer decorator

def timer(func):
    """Decorator that times function execution."""
    @wraps(func)  # Preserves func's __name__ and __doc__
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)  # Call the original function
        elapsed = time.time() - start
        print(f"{func.__name__} took {elapsed:.4f}s")
        return result
    return wrapper

@timer
def slow_function():
    """A function that takes a while."""
    time.sleep(0.1)
    return "done"

result = slow_function()  # Prints: slow_function took 0.1xxxs
print(result)  # done
print(slow_function.__name__)  # slow_function (preserved by @wraps)

In [ ]:
import time
from functools import wraps

# Real-world: @retry decorator
def retry(max_attempts=3, delay=0.1):
    """Decorator factory that retries on failure."""
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            for attempt in range(1, max_attempts + 1):
                try:
                    return func(*args, **kwargs)
                except Exception as e:
                    if attempt == max_attempts:
                        raise
                    print(f"Attempt {attempt} failed: {e}. Retrying...")
                    time.sleep(delay)
        return wrapper
    return decorator

# Use it
attempt_count = 0

@retry(max_attempts=3, delay=0.01)
def flaky_operation():
    global attempt_count
    attempt_count += 1
    if attempt_count < 3:
        raise ConnectionError("Server unavailable")
    return "Success!"

print(flaky_operation())

### Exercise 3.1: Write a @log_call decorator
That prints the function name, arguments, and return value every time the function is called.

In [ ]:
# Exercise 3.1
# YOUR CODE HERE
from functools import wraps


---
## Section 4: Generators

A **generator** is a function that produces values one at a time using `yield`. 
Unlike a regular function that computes everything at once and returns a list, 
a generator is lazy -- it only computes the next value when asked.

Why? **Memory efficiency.** If you have 10 million rows, you do not want to load 
them all into memory at once. A generator lets you process them one batch at a time.

In [ ]:
# Regular function: builds entire list in memory
def get_squares_list(n):
    result = []
    for i in range(n):
        result.append(i ** 2)
    return result

# Generator function: yields one value at a time
def get_squares_gen(n):
    for i in range(n):
        yield i ** 2  # Pauses here, resumes when next value is requested

# Both produce the same values
print(list(get_squares_list(5)))  # [0, 1, 4, 9, 16]
print(list(get_squares_gen(5)))   # [0, 1, 4, 9, 16]

# But the generator does not build the full list!
gen = get_squares_gen(5)
print(next(gen))  # 0
print(next(gen))  # 1
print(next(gen))  # 4 -- it picks up where it left off

In [ ]:
# Practical: chunk reader (process data in batches)
def chunk_reader(data, chunk_size=3):
    """Yield data in chunks of chunk_size."""
    for i in range(0, len(data), chunk_size):
        yield data[i:i + chunk_size]

all_rows = list(range(10))  # [0, 1, 2, ..., 9]

for batch_num, chunk in enumerate(chunk_reader(all_rows, 3), 1):
    print(f"Batch {batch_num}: {chunk}")
# Batch 1: [0, 1, 2]
# Batch 2: [3, 4, 5]
# Batch 3: [6, 7, 8]
# Batch 4: [9]

### Generator expressions

Like list comprehensions, but with parentheses instead of brackets. 
They create generators instead of lists.

```python
squares_list = [x**2 for x in range(1000000)]  # Uses memory for ALL values
squares_gen  = (x**2 for x in range(1000000))  # Uses almost no memory
```

### Exercise 4.1: Write a Fibonacci generator
Write a generator that yields Fibonacci numbers: 0, 1, 1, 2, 3, 5, 8, 13, ...
Print the first 10 Fibonacci numbers.

In [ ]:
# Exercise 4.1
# YOUR CODE HERE


---
## Section 5: Context Managers

You know `with open(...) as f:` -- that is a context manager. 
Now you will build your own.

A context manager handles setup and cleanup automatically. 
Odibi's `PhaseTimer` in `node.py` uses this exact pattern to time execution phases.

In [ ]:
from contextlib import contextmanager
import time

# The easy way: @contextmanager decorator
@contextmanager
def phase_timer(phase_name):
    """Time a phase of execution (like Odibi PhaseTimer)."""
    print(f"Starting: {phase_name}")
    start = time.time()
    yield  # Everything inside the 'with' block runs here
    elapsed = time.time() - start
    print(f"Completed: {phase_name} in {elapsed:.3f}s")

# Use it
with phase_timer("read"):
    time.sleep(0.1)  # Simulate reading data

with phase_timer("transform"):
    time.sleep(0.05)  # Simulate transformation

### Exercise 5.1: Temporary config context manager
Write a `@contextmanager` called `temp_config` that temporarily overrides a dict value 
and restores it when the block ends.

In [ ]:
# Exercise 5.1
# YOUR CODE HERE
from contextlib import contextmanager




# Test:
# config = {"engine": "pandas", "debug": False}
# with temp_config(config, "debug", True):
#     print(f"Inside: debug={config['debug']}")  # True
# print(f"Outside: debug={config['debug']}")  # False (restored)

---
## Section 6: Lambda, map, filter

`lambda` creates a small anonymous function in one line:
```python
lambda x: x * 2     # Same as: def double(x): return x * 2
```

Use lambda for:
- Sorting keys: `sorted(items, key=lambda x: x['name'])`
- Simple callbacks

Do NOT use lambda for anything complex. If it does not fit on one line, use `def`.

In [ ]:
# Lambda examples
nodes = [
    {"name": "customers", "rows": 1542, "duration": 3.45},
    {"name": "orders", "rows": 8930, "duration": 12.8},
    {"name": "products", "rows": 234, "duration": 1.2},
]

# Sort by rows (descending)
by_rows = sorted(nodes, key=lambda n: n["rows"], reverse=True)
print([n["name"] for n in by_rows])  # ["orders", "customers", "products"]

# Sort by duration
by_time = sorted(nodes, key=lambda n: n["duration"])
print([f"{n["name"]}: {n["duration"]}s" for n in by_time])

# map() - apply function to every item
names = ["alice", "bob", "charlie"]
upper = list(map(str.upper, names))
print(upper)  # Prefer: [n.upper() for n in names]

---
## Section 7: Interview Drill


### Drill 1: Write a decorator from scratch
Write a `@count_calls` decorator that tracks how many times a function has been called.

In [ ]:
# Drill 1
# YOUR CODE HERE


### Drill 2: What is the difference between a generator and a list?
Write code that demonstrates the memory difference.

In [ ]:
# Drill 2
# YOUR CODE HERE


### Drill 3: Write a context manager for database connections
That prints 'Connecting...' on entry and 'Disconnecting...' on exit.

In [ ]:
# Drill 3
# YOUR CODE HERE


---
## Checkpoint

You now know Python's advanced patterns:
- First-class functions, closures
- Decorators (with and without arguments)
- Generators and yield (lazy evaluation)
- Context managers (@contextmanager)
- Lambda, map, filter

**Next:** Phase 6 -- Pydantic and Type Safety. This is where you start building 
mini-odibi's configuration system.